In [1]:
# Mount google drive

from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

import os

path = "/content/gdrive/MyDrive/BANA6350/Data"

os.chdir(path)

Mounted at /content/gdrive/


In [2]:
import matplotlib.pyplot as plt

plt.rcParams['lines.linewidth'] = 3
plt.rcParams['figure.figsize'] = [14.0, 6.0]
plt.rcParams['font.size']= 18
plt.style.available   # Check what styles are available for Chart formats by visiting : https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
plt.style.use('fivethirtyeight')       # Assigning the FiveThirtyEight format, you can choose any of the names from the above link

In [3]:
%matplotlib inline

In [4]:
!pip install dmba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 46.2 MB/s eta 0:00:00


In [5]:
!pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 16.6 MB/s eta 0:00:00


In [6]:
# importing standard libraries

import math
from pathlib import Path
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import from_levels_and_colors
import seaborn as sns

# loading specific libraries for this class

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import power

from dmba import stepwise_selection
from dmba import AIC_score

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (confusion_matrix, accuracy_score)
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression #, LogisticRegressionCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score

import patsy

Colab environment detected.


In [36]:
df = pd.read_csv('titanic.csv')
df.dropna(inplace = True)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
869,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
870,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
877,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
885,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 887
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB


In [38]:
numerical = ['Age', 'Fare', 'SibSp', 'Parch']
X_numerical = df[numerical]

In [39]:
X_categorical = pd.get_dummies(df[['Sex']])
X = pd.concat([X_numerical, X_categorical], axis = 1)
X

,Age,Fare,SibSp,Parch,Sex_female,Sex_male
1,38.0,71.2833,1,0,1,0
3,35.0,53.1000,1,0,1,0
6,54.0,51.8625,0,0,0,1
10,4.0,16.7000,1,1,1,0
11,58.0,26.5500,0,0,1,0
...,...,...,...,...,...,...
869,47.0,52.5542,1,1,1,0
870,33.0,5.0000,0,0,0,1
877,56.0,83.1583,0,1,1,0
885,19.0,30.0000,0,0,1,0


In [40]:
y = df['Survived']
y

1      1
3      1
6      0
10     1
11     1
      ..
869    1
870    0
877    1
885    1
887    1
Name: Survived, Length: 183, dtype: int64

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=42)

In [42]:
log_reg = sm.Logit(y_train, X_train).fit()
print(log_reg.summary())
print(log_reg.summary2())

Optimization terminated successfully.
         Current function value: 0.445487
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  146
Model:                          Logit   Df Residuals:                      140
Method:                           MLE   Df Model:                            5
Date:                Sun, 03 Mar 2024   Pseudo R-squ.:                  0.2850
Time:                        21:39:42   Log-Likelihood:                -65.041
converged:                       True   LL-Null:                       -90.972
Covariance Type:            nonrobust   LLR p-value:                 5.758e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Age           -0.0231      0.015     -1.578      0.114      -0.052       0.006
Fare           0.0045      0.

In [43]:
predictions = log_reg.predict(X_test)
predictions = list(map(round, predictions))
print('Actual values', list(y_test.values))
print('Predictions :', predictions)

Actual values [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1]
Predictions : [1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]


In [44]:
from sklearn.metrics import (confusion_matrix, accuracy_score)
print('Test accuracy for Model 1 = ', accuracy_score(y_test, predictions))

Test accuracy for Model 1 =  0.7567567567567568


In [ ]:
#Model 2

In [47]:
Xtrain = df[['Age', 'Fare', 'SibSp', 'Parch']]
Xtrain

,Age,Fare,SibSp,Parch
1,38.0,71.2833,1,0
3,35.0,53.1000,1,0
6,54.0,51.8625,0,0
10,4.0,16.7000,1,1
11,58.0,26.5500,0,0
...,...,...,...,...
869,47.0,52.5542,1,1
870,33.0,5.0000,0,0
877,56.0,83.1583,0,1
885,19.0,30.0000,0,0


In [48]:
ytrain = df[['Survived']]
ytrain

,Survived
1,1
3,1
6,0
10,1
11,1
...,...
869,1
870,0
877,1
885,1


In [49]:
log_reg = sm.Logit(ytrain, Xtrain).fit()

Optimization terminated successfully.
         Current function value: 0.622294
         Iterations 6


In [50]:
print(log_reg.summary())
print(log_reg.summary2())

                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  183
Model:                          Logit   Df Residuals:                      179
Method:                           MLE   Df Model:                            3
Date:                Sun, 03 Mar 2024   Pseudo R-squ.:                 0.01638
Time:                        21:43:19   Log-Likelihood:                -113.88
converged:                       True   LL-Null:                       -115.78
Covariance Type:            nonrobust   LLR p-value:                    0.2846
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Age           -0.0040      0.006     -0.709      0.478      -0.015       0.007
Fare           0.0081      0.003      2.422      0.015       0.002       0.015
SibSp          0.3972      0.296      1.342      0.1

In [51]:
Xtest = df[['Age', 'Fare', 'SibSp', 'Parch']]
ytest = df['Survived']

In [52]:
prediction = log_reg.predict(Xtest)

In [55]:
prediction = list(map(round, predictions))


In [56]:
print('Actual values', list(ytest.values))
print('Predictions :', prediction)

Actual values [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
Predictions : [1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]


In [57]:
print('Test accuracy = ', accuracy_score(ytest, prediction))

ValueError: Found input variables with inconsistent numbers of samples: [183, 37]